In [1]:
import pandas as pd
import zipfile
import os
import shutil
import json

In [2]:
all_lang_labels = pd.read_excel("Dashbord Labels Translations.xlsx")
all_lang_labels["English"] = all_lang_labels["English"].apply(lambda x: x.strip())
all_lang_labels.columns

Index(['English', 'Japanese', 'Bahasa Malaysia'], dtype='object')

In [3]:
for LanguageName in all_lang_labels.columns[1:]:
    # if(LanguageName in ["Korean"]):
    print(LanguageName)

    #Extracting Layout from PBIT file for each app

    # Convert .pbit to .zip
    pbit_file = 'Multilingual Dashboard_TestApp.pbit'
    zip_file = "Multilingual Dashboard_TestApp_"+LanguageName+".zip"
    os.rename(pbit_file, zip_file)

    

    # Extract .zip file
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        extract_folder = "Multilingual Dashboard_TestApp_"+LanguageName+"_Extracted"
        zip_ref.extractall(extract_folder)

    #again renaming just to avoid loosing app
    os.rename(zip_file,pbit_file)    


    # load Layout file
    import json
    filepath = extract_folder+"\Report\Layout"
    print(filepath)
    with open(filepath, "r", encoding = 'utf-16 le') as f:
        orgdata = json.load(f)
    data = orgdata.copy() 


    #removing extra space
    all_lang_labels[LanguageName] = all_lang_labels[LanguageName].apply(lambda x: x.strip())

    #Objectc title mapping with new language
    ExcelFile = "Label_output.xlsx"
    ObjecLable = pd.read_excel(ExcelFile, sheet_name=None)
    for sheetname,ExcelRowData in ObjecLable.items():
        # print(sheetname)
        for  index,row in ExcelRowData.iterrows():
            if(row["Label"].strip() in list(all_lang_labels["English"])):
                row["New Label"] = all_lang_labels[row["Label"].strip() == all_lang_labels["English"]][LanguageName].values[0]
                # print(row)
            else:
                row["New Label"] = row["Label"].strip()+"_missing"

        with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
            # Write the updated ExcelRowDataframe to Sheet1 
            ExcelRowData.to_excel(writer, sheet_name=sheetname, index=False) 


    #Table column name mapping with new language
    ExcelFile = "TableLableExtracted.xlsx"
    ObjecLable = pd.read_excel(ExcelFile, sheet_name=None)
    for sheetname,ExcelRowData in ObjecLable.items():
        # print(sheetname)
        for  index,row in ExcelRowData.iterrows():
            if(row["Label"].strip() in list(all_lang_labels["English"])):
                row["New Label"] = all_lang_labels[row["Label"].strip() == all_lang_labels["English"]][LanguageName].values[0]
                # print(row)
            else:
                row["New Label"] = row["Label"].strip()+"_missing"

        with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
            # Write the updated ExcelRowDataframe to Sheet1 
            ExcelRowData.to_excel(writer, sheet_name=sheetname, index=False)  

    #Bar chart and piechart column name mapping with new language
    ExcelFile = "BarchartLableExtracted.xlsx"
    ObjecLable = pd.read_excel(ExcelFile, sheet_name=None)
    for sheetname,ExcelRowData in ObjecLable.items():
        # print(sheetname)
        for  index,row in ExcelRowData.iterrows():
            if(row["Label"].strip() in list(all_lang_labels["English"])):
                row["New Label"] = all_lang_labels[row["Label"].strip() == all_lang_labels["English"]][LanguageName].values[0]
                # print(row)
            else:
                row["New Label"] = row["Label"].strip()+"_missing"

        with pd.ExcelWriter(ExcelFile, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer: 
            # Write the updated ExcelRowDataframe to Sheet1 
            ExcelRowData.to_excel(writer, sheet_name=sheetname, index=False)   

    # Update all object labels
    excel_file = 'Label_output.xlsx'
    exceldata = pd.read_excel(excel_file, sheet_name=None)
    position = 0
    for (sheet_name, exceldf), jsondata in zip(exceldata.items(), data["sections"]): 
        #pull excel data sheet wise and remove null rows
        exceldf = exceldf[~exceldf["Label"].isna()].reset_index(drop=True)
        # exceldf.set_index("ID",inplace=True)
        # print(exceldf)
        jsonsheetname = jsondata["displayName"] 
        # print(exceldf.index.to_list())

        if(sheet_name == jsonsheetname):
            innerposition = 0
            for j in jsondata["visualContainers"]:

                content = j["config"]
                objectid = content
                objectid = objectid[9:29]
                if(len(exceldf[exceldf["ID"] == objectid])>0):
                    NewLabel = list(exceldf[exceldf["ID"] == objectid]["New Label"])[0]
                    Label = list(exceldf[exceldf["ID"] == objectid]["Label"])[0]
                    # print(Label,NewLabel)
                    data["sections"][position]["visualContainers"][innerposition]["config"] = data["sections"][position]["visualContainers"][innerposition]["config"].replace(Label,NewLabel)
                innerposition += 1    
            position += 1     


    #Label update for all tables
    excel_file = 'TableLableExtracted.xlsx'
    for i in data["sections"]:
        currentsheet = i["displayName"]

        for j in i["visualContainers"]:
            content = json.loads(j["config"])

            if(content["singleVisual"]["visualType"]=="tableEx"):
                exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)

                for key,value in content["singleVisual"]["columnProperties"].items():
                    # print(currentsheet)
                    # if(currentsheet == "SSR vs RTD"):
                    
                    # print(checkid)
                    # print(list(exceldf[exceldf["ID"] == checkid]["New Label"])[0] )
                    # print(checkid)
                    try:
                        checkid = (key+content["name"]+value["displayName"])
                        newvalue = list(exceldf[exceldf["ID"] == checkid]["New Label"])[0]
                        # print(value["displayName"])
                        value["displayName"] = newvalue

                    except:
                        # print("here")
                        pass
            content_json_string = json.dumps(content)
            j["config"] = content_json_string     


    #Label update for all bar and donut charts
    excel_file = 'BarchartLableExtracted.xlsx'
    for i in data["sections"]:
        currentsheet = i["displayName"]

        for j in i["visualContainers"]:
            content = json.loads(j["config"])

            if(content["singleVisual"]["visualType"] in ["clusteredColumnChart","clusteredBarChart","barChart","donutChart","columnChart"]):
                exceldf = pd.read_excel(excel_file,sheet_name=currentsheet)
                # print(currentsheet)
                for key,value in content["singleVisual"]["columnProperties"].items():
                    try:
                        checkid = (key+content["name"]+value["displayName"])
                        newvalue = list(exceldf[exceldf["ID"] == checkid]["New Label"])[0]
                        value["displayName"] = newvalue
                    except:
                        pass

            content_json_string = json.dumps(content)
            j["config"] = content_json_string   

    #dumping with updated labels
    with open(filepath, 'w', encoding = 'utf-16 le') as outfile:
        json.dump(data, outfile, indent = 4)

    #removing security file 
    os.remove(extract_folder+"\SecurityBindings")
    os.getcwd()    

    #Repack extracted folder to .zip
    # extract_folder = 'Multilingual Dashboard_TestApp Extracted'
    shutil.make_archive('Multilingual Dashboard_TestApp_'+LanguageName, 'zip', extract_folder)
    #Rename .zip to .pbit
    new_zip_file = "Multilingual Dashboard_TestApp_"+LanguageName+".zip"
    new_pbit_file = "Multilingual Dashboard_TestApp_"+LanguageName+".pbit"
    os.rename(new_zip_file, new_pbit_file)

    # connection
    # ecl-ecosure-tv-sql-001-s.database.windows.net
    # TrueViewDWDB                            



        
    

    

Japanese
Multilingual Dashboard_TestApp_Japanese_Extracted\Report\Layout
Bahasa Malaysia
Multilingual Dashboard_TestApp_Bahasa Malaysia_Extracted\Report\Layout
